In [1]:
import tensorflow as tf
import tensorflow_hub as hub

2022-09-16 22:21:27.582576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 22:21:27.672137: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-16 22:21:27.672153: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-09-16 22:21:27.690209: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-16 22:21:31.546215: W tensorflow/stream_executor/platform/de

In [17]:
import pandas as pd
df=pd.read_csv("textspam.csv")
df.head(5)

,Category,Message
0,safe,Asante mamy
1,safe,Haya
2,spam,Uniwekee kwa tigopesa 0657586575 Jina KA...
3,safe,Pole sana
4,spam,KARIBU FREEMASON UMILIKI PESA MAGARI MAJUMBA B...


In [18]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
safe           9      9                                        Asante mamy   
spam           5      5    Uniwekee  kwa   tigopesa 0657586575 Jina  KA...   

               
         freq  
Category       
safe        1  
spam        1

In [19]:
df['Category'].value_counts()

safe    9
spam    5
Name: Category, dtype: int64

In [21]:
df_spam = df[df['Category']== 'spam']
df_spam.shape

(5, 2)

In [22]:
df_ham = df[df['Category']== 'safe']
df_ham.shape

(9, 2)

In [23]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])
df_ham_downsampled.shape

(5, 2)

In [24]:
df_balanced = pd.concat([df_spam, df_ham_downsampled])
df_balanced.shape

(10, 2)

In [25]:
df_balanced['Category'].value_counts()

spam    5
safe    5
Name: Category, dtype: int64

In [26]:
df_balanced.sample(5)

,Category,Message
1,safe,Haya
4,spam,KARIBU FREEMASON UMILIKI PESA MAGARI MAJUMBA B...
6,spam,itume Tigopesa 0715379261 jina MARICK SEV...
0,safe,Asante mamy
11,safe,Mambo vp !!Shani naomba nisaidie elfu 5 me pepe


In [27]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)

,Category,Message,spam
4,spam,KARIBU FREEMASON UMILIKI PESA MAGARI MAJUMBA B...,1
8,spam,MPIGI£ MZ££ SIMBA KWA T!BA ASIL!A PETE KESI MA...,1
7,spam,Kama unaweza wewe Njoo uchukue cash. Sina hela...,1
11,safe,Mambo vp !!Shani naomba nisaidie elfu 5 me pepe,0
2,spam,Uniwekee kwa tigopesa 0657586575 Jina KA...,1


In [28]:
df_balanced['spam'] = df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)
df_balanced.sample(5)


,Category,Message,spam
2,spam,Uniwekee kwa tigopesa 0657586575 Jina KA...,1
1,safe,Haya,0
5,safe,Piga mwenyewe,0
4,spam,KARIBU FREEMASON UMILIKI PESA MAGARI MAJUMBA B...,1
0,safe,Asante mamy,0


In [30]:
from sklearn.model_selection import train_test_split
x_train, y_train, x_test, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

In [31]:
x_train.head(4)

11      Mambo vp !!Shani naomba nisaidie elfu 5 me pepe
5                                         Piga mwenyewe
4     KARIBU FREEMASON UMILIKI PESA MAGARI MAJUMBA B...
13                                   Nashukuru Shangazi
Name: Message, dtype: object

In [32]:
import tensorflow_text as text
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

2022-09-16 22:43:48.456238: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-09-16 22:43:48.456366: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-16 22:43:48.456433: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (egovridc-Vostro-3888): /proc/driver/nvidia/version does not exist
2022-09-16 22:43:48.523311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-16 22:43:56.880052: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 937635

In [33]:
def get_sentence_embeding(sentences):
    prepocessed_text = bert_preprocess(sentences)
    return bert_encoder(prepocessed_text)['pooled_output']

get_sentence_embeding([
    "niwekee kwa tigopesa",
    "Nashukuru Shangazi"
])

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.8829554 , -0.5831021 , -0.72702557, ..., -0.63066083,
        -0.6589997 ,  0.7883161 ],
       [-0.8832398 , -0.37036562, -0.62820566, ..., -0.43516645,
        -0.5521083 ,  0.8363966 ]], dtype=float32)>

In [34]:
e = get_sentence_embeding([
    "banana", 
    "dodoma",
    "mango",
    "tuma humu",
    "ile pesa",
    "ngoswe"
]
)

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]], [e[1]])

array([[0.73551345]], dtype=float32)

In [36]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

L =tf.keras.layers.Dropout(0.1, name='dropout')(outputs['pooled_output'])
L =tf.keras.layers.Dense(1, activation='sigmoid', name='output')(L)

model = tf.keras.Model(inputs=[text_input], outputs=[L])

In [37]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [38]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [43]:
model.fit(x_train, y_train, epochs=1)

ValueError: Data cardinality is ambiguous:
  x sizes: 7
  y sizes: 3
Make sure all arrays contain the same number of samples.